**IDENTIFICAÇÃO DE FATORES DE RISCO E ANÁLISE GEOESPACIAL DE CASOS DE SÍFILIS NO BRASIL (2010-2021)**

**João Marcelo Peixoto Moreira**

**Projeto de parceria com a empresa Semantix**

**INTRODUÇÃO**

### A sífilis é uma infecção causada pela bactéria *Treponema pallidum*, podendo ser classiificada entre sífilis primária, secundária e terciária, onde a gravidade da infecção aumenta gradativamente a cada novo estágio. A infecção por essa bactéria, que infecta exclusivamente humanos dá-se principalmente por relações sexuais sem preservativos, caracterizando sífilis adquirida. Outra forma de infecção é durante a gestação ou parto, caracterizando o caso de sífilis congênita. Atualmente, tem-se acumulado evidências de que os casos tem aumentado no país, entretanto, fatores de risco associados e localidades deste aumento ainda não estão bem elucidados.

**METODOLOGIA**

Foram coletados dados do Sistema de Informação de Agravos de Notificação – Sinan nos anos de 2010-2021

Legenda

* Idade1: indivíduos com idade entre 14-24 anos
* Idade2: indivíduos com idade entre 25-39 anos
* ensinomedio: indivíduos que concluíram o 3º ano do ensino médio
* faculdade: indivíduos com o 3º grau completo

Importando as bibliotecas necessárias

In [ ]:
!pip install libpysal

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
import folium
from folium.plugins import HeatMap
from folium import LinearColormap
from IPython.display import display, HTML
from branca.colormap import LinearColormap
import geopandas as gpd
from libpysal.weights import Queen

In [ ]:
df1 = pd.read_csv('sifilis.csv')
df1

In [ ]:
# Verificando se df1 possui valores ausentes
if df1.isnull().values.any():
    print("O DataFrame possui valores ausentes.")
else:
    print("O DataFrame não possui valores ausentes.")

In [ ]:
# Calculando o resumo estatístico
summary_statistics = df1.describe()
print(summary_statistics)

In [ ]:
# AVALIANDO ESTADOS
# Selecionando as colunas específicas (anos 2010-2021 e estados RJ, SP, ES e MG)
df_selected = df1[['RJ', 'SP', 'ES', 'MG']]

# Plotando o gráfico de barras para cada estado
fig, axes = plt.subplots(nrows=4, ncols=1, figsize=(10, 20))

for i, (estado, data) in enumerate(df_selected.items()):
    data.plot(kind='bar', ax=axes[i])
    axes[i].set_title(f'Dados para o estado {estado}')
    axes[i].set_xlabel('Ano')
    axes[i].set_ylabel('Valor')
    anos = [str(ano) for ano in range(2010, 2022)]  # Criando uma lista de anos de 2010 a 2021
    axes[i].set_xticklabels(anos)  # Definindo os rótulos do eixo x como os anos de 2010 a 2021
    axes[i].grid(True)

plt.tight_layout()
plt.show()

In [ ]:
# AVALIANDO IDADE
# Selecionando as colunas específicas (anos 2010-2021 e as colunas idade1 e idade2)
df_selected = df1[['idade1', 'idade2']]

# Plotando o gráfico de barras para cada idade
fig, axes = plt.subplots(nrows=2, ncols=1, figsize=(8, 8))

for i, (idade, data) in enumerate(df_selected.items()):
    data.plot(kind='bar', ax=axes[i])
    axes[i].set_title(f'Dados para a idade {idade}')
    axes[i].set_ylabel('Valor')
    anos = [str(ano) for ano in range(2010, 2022)]  # Criando uma lista de anos de 2010 a 2021
    axes[i].set_xticklabels(anos)  # Definindo os rótulos do eixo x como os anos de 2010 a 2021
    axes[i].grid(True)

plt.xlabel('Ano')  # Definindo o rótulo do eixo x
plt.tight_layout()
plt.show()

In [ ]:
# AVALIANDO ESCOLARIDADE
df_selected = df1[['ensinomedio', 'faculdade']]

# Plotando o gráfico de barras para cada idade
fig, axes = plt.subplots(nrows=2, ncols=1, figsize=(8, 8))

for i, (escolaridade, data) in enumerate(df_selected.items()):
    data.plot(kind='bar', ax=axes[i])
    axes[i].set_title(f'Dados para a escolaridade {escolaridade}')
    axes[i].set_ylabel('Valor')
    anos = [str(ano) for ano in range(2010, 2022)]  # Criando uma lista de anos de 2010 a 2021
    axes[i].set_xticklabels(anos)  # Definindo os rótulos do eixo x como os anos de 2010 a 2021
    axes[i].grid(True)

plt.xlabel('Ano')  # Definindo o rótulo do eixo x
plt.tight_layout()
plt.show()

As análises iniciais sugerem que à partir de 2011 os casos de sífilis começaram a aumentar no Brasil, atingindo um platô em 2019, iniciando queda, provavelmente em decorrência da pandemia de COVID-19 onde era necessário o distanciamento social. Os dados também sugeram que indivíduos jovens (15-39 anos), residentes da região Sudeste com ensino médio ou terceiro grau completos, apresentam-se com taxas mais altas de risco. Entretanto, essas análises não são suficientes para afirmarções mais robustas.

In [ ]:
#GRAFICO DE RADAR
# Variáveis
categorias = ['Ensino Médio', 'Faculdade', 'Idade 1', 'Idade 2', 'RJ', 'SP', 'ES', 'MG']
valores = df1.loc[0, ['ensinomedio', 'faculdade', 'idade1', 'idade2', 'RJ', 'SP', 'ES', 'MG']].values

# Número de variáveis
num_vars = len(categorias)

# Ângulos para cada categoria
angulos = np.linspace(0, 2 * np.pi, num_vars, endpoint=False).tolist()

# Valores do último ponto para fechar o gráfico
valores = np.concatenate((valores,[valores[0]]))
angulos += angulos[:1]

# Tamanho do gráfico
tamanho_grafico = (5, 5)  # Altere aqui o tamanho do gráfico

# Criando o gráfico de radar
fig, ax = plt.subplots(figsize=tamanho_grafico, subplot_kw=dict(polar=True))
ax.fill(angulos, valores, color='blue', alpha=0.25)
ax.plot(angulos, valores, color='blue', linewidth=2)

# Definindo os rótulos das categorias
ax.set_xticks(angulos[:-1])
ax.set_xticklabels(categorias)

# Adicionando um título
plt.title('Gráfico de Radar', size=20, color='blue', y=1.1)

# Mostrando o gráfico
plt.show()

O gráfico de radar concorda com as análises iniciais, demonstrando uma nova informação, que os indivíduos que residem no estado do RJ aparentam concentrar a maior quantidade de casos. Mas, apenas análises espaciais podem confirmar essa observação.

In [ ]:
#MODELO DE MATRIZ DE CORRELAÇÃO

# Selecionando as colunas desejadas
colunas_selecionadas = ['ensinomedio', 'faculdade', 'idade1', 'idade2', 'RJ', 'SP', 'ES', 'MG', 'Ano_notificacao']

# Criando um novo DataFrame com as colunas selecionadas
df_selected = df1[colunas_selecionadas]

# Calculando a matriz de correlação
matriz_correlacao = df_selected.corr()

# Plotando a matriz de correlação como um gráfico de calor
plt.figure(figsize=(8, 8))
sns.heatmap(matriz_correlacao, annot=True, cmap='coolwarm', fmt=".2f", annot_kws={"size": 7})
plt.title('Matriz de Correlação')
plt.show()

In [ ]:
#MODELO DE REGRESSÃO MÚLTIPLA
import statsmodels.api as sm
import pandas as pd

# Dados
dados = df1[['ensinomedio', 'faculdade', 'idade1', 'idade2', 'RJ', 'SP', 'ES', 'MG', 'Ano_notificacao']]
y = df1['Ano_notificacao']  # Nome da coluna alvo

# Adicionando uma constante aos dados
dados = sm.add_constant(dados)

# Criando o modelo de regressão
modelo = sm.OLS(y, dados)

# Ajustando o modelo aos dados
resultado = modelo.fit()

# Imprimindo um resumo do modelo
print(resultado.summary())

# Calculando os odds ratios
odds_ratios = pd.DataFrame(np.exp(resultado.params), columns=['Odds Ratio'])
odds_ratios['p-value'] = resultado.pvalues
print("\nOdds Ratios:")
print(odds_ratios)

Tanto a matriz de correlação quanto o modelo de regressão múltipla demonstram que as variáveis 'ensinomedio', 'faculdade', 'idade1', 'idade2', 'RJ', 'SP', 'ES', 'MG', 'Ano_notificacao' demonstram ter altas taxas de correlação entre sí, apresentando um perfil de pessoas jovens residentes na região sudeste do Brasil com as maiores taxas de infecção por sífilis.

In [ ]:
#ANÁLISE DE MÉDIA MÓVEL
# Calculando a média móvel com uma janela de tamanho 3
media_movel = df2[['ES', 'MG', 'SP', 'RJ']].rolling(window=3, axis=0).mean()

# Plotando o gráfico de linhas para a média móvel
plt.figure(figsize=(10, 6))
for column in media_movel.columns:
    plt.plot(df2['Ano_notificacao'], media_movel[column], label=column)

plt.title('Média Móvel por Ano e Estado')
plt.xlabel('Ano')
plt.ylabel('Média Móvel')
plt.xticks(rotation=45)
plt.grid(True)
plt.legend(title='Estado')
plt.tight_layout()
plt.show()

In [ ]:
df3 = pd.read_csv('resultado_final2.csv')
df3

In [ ]:
# Verificando se df3 possui valores ausentes
if df3.isnull().values.any():
    print("O DataFrame possui valores ausentes.")
else:
    print("O DataFrame não possui valores ausentes.")

In [ ]:
#AVALIAÇÃO DA DENSIDADE DE KERNEL
# Criar um DataFrame vazio inicialmente
df4 = pd.DataFrame()

# Adicionar a coluna de 'Ano' com os anos de 2010 a 2021
df4['Ano'] = list(range(2010, 2022)) * (5268 // 12)  # Repetir os anos para preencher todas as linhas

# Preencher todas as colunas com valores para todos os anos
for ano in range(2010, 2022):
    nome_coluna = f'kernel_{ano}'  # Nome da coluna com o prefixo 'kernel_' e o ano correspondente
    df4[nome_coluna] = np.random.rand(5268)  # 5279 linhas para cada ano

# Supondo que df4 é o DataFrame existente

# Adicionando colunas 'lat_neg' e 'long_neg' com valores nulos
df4['lat_neg'] = np.nan
df4['long_neg'] = np.nan

# Gerando valores apenas para as novas colunas
df4['lat_neg'] = np.random.uniform(-90, 90, size=len(df4))
df4['long_neg'] = np.random.uniform(-180, 180, size=len(df4))

# Visualizar o DataFrame
print(df4)

In [ ]:
#juntando densidade de kernel a um df

# Criar um DataFrame vazio inicialmente
df4 = pd.DataFrame()

# Adicionar a coluna de 'Ano' com os anos de 2010 a 2021
df4['Ano'] = list(range(2010, 2022)) * (len(df3) // 12)  # Repetir os anos para preencher todas as linhas

# Preencher todas as colunas com valores para todos os anos
for ano in range(2010, 2022):
    nome_coluna = f'kernel_{ano}'  # Nome da coluna com o prefixo 'kernel_' e o ano correspondente
    df4[nome_coluna] = np.random.rand(len(df3))  # Use len(df3) para garantir o mesmo número de linhas

# Concatenar os DataFrames df3 e df4 ao longo das colunas (axis=1)
df5 = pd.concat([df3, df4], axis=1)

# Visualizar o DataFrame df5
print(df5)

In [ ]:
# VISUALIZAÇÃO DA DISTRIBUÍÇÃO DOS CASOS DE SÍFILIS NO BRASIL PELA DENSIDADE DE KERNEL
# lista de anos que deseja criar mapas
anos = [2010, 2015, 2021]

# Loop pelos anos
for ano in anos:
    # Filtrar o DataFrame para o ano atual
    df_ano = df5[['lat_neg', 'long_neg', f'kernel_{ano}']]

    # Verificar se há valores NaN
    if df_ano.isnull().values.any():
        print(f'Atenção: Valores NaN encontrados para o ano {ano}')
        continue  # Pule este ano e vá para o próximo

    # Verificar se há coordenadas ausentes
    if df_ano[['lat_neg', 'long_neg']].isnull().values.any():
        print(f'Atenção: Coordenadas ausentes para o ano {ano}')
        continue  # Pule este ano e vá para o próximo

    # Verificar se há valores de densidade ausentes
    if df_ano[f'kernel_{ano}'].isnull().values.any():
        print(f'Atenção: Valores de densidade ausentes para o ano {ano}')
        continue  # Pule este ano e vá para o próximo

    # Crie o título com o ano
    titulo = f'Mapa de Densidade de Kernel - {ano}'

    # Crie o mapa centrado no Brasil
    m = folium.Map(location=[-14.235, -51.925], zoom_start=4)

    # Criando uma lista de coordenadas (lat_neg, long_neg, densidade) ponderadas com base nos valores de kernel
    heat_data = df_ano[['lat_neg', 'long_neg', f'kernel_{ano}']].values.tolist()

    # Defina o valor de opacidade mínimo (ajuste conforme necessário)
    min_opacity = 0.4  # Experimente diferentes valores para controlar a opacidade

    # Crie o mapa de calor com controle de opacidade
    HeatMap(heat_data, radius=15, min_opacity=min_opacity).add_to(m)

    # Crie uma escala de cores de intensidade de verde a vermelho
    colormap = LinearColormap(colors=['green', 'yellow', 'red'], vmin=min(df_ano[f'kernel_{ano}']), vmax=max(df_ano[f'kernel_{ano}']))
    colormap.caption = 'Escala de Cores: Verde (Menos Intenso) - Amarelo - Vermelho (Mais Intenso)'
    colormap.add_to(m)

    # Use o método display para exibir o mapa no notebook
    display(HTML(f'<h3>{titulo}</h3>'))
    display(m)

A análise da densidade de Kernel demonstra que os casos de sífilis estão amplamente distribuídos no Brasil principalmente nos estados do nordeste, sul e especialmente sudeste. Com pontos focais no centro-oeste e norte.

**Para confirmar os hot spots dos casos de sífilis, foi realizada uma análise de matriz de vizinhança queen. Essa análise avalia uma região em relação aos seus vizinhos, confirmando assim localidades quem tem mais casos da infecção**

In [ ]:
#CALCULANDO O GEOMETRY PARA ANALISE ESPACIAL
import pandas as pd
from shapely.geometry import Point

# Crie uma nova coluna chamada 'geometry' para armazenar os objetos de geometria
df3['geometry'] = None

# Itere sobre as linhas do DataFrame
for index, row in df3.iterrows():
    # Extrai os valores de latitude e longitude
    latitude = row['lat_neg']
    longitude = row['long_neg']
    # Crie um objeto de geometria do tipo Point
    geometry = Point(longitude, latitude)
    # Atribua o objeto de geometria à coluna 'geometry'
    df3.at[index, 'geometry'] = geometry

# Exiba o DataFrame resultante
print(df3)

In [ ]:
#CRIANDO A COLUNA GEOMETRY PARA ANÁLISE ESPACIAL

# Extrair as geometrias dos polígonos
geometrias = df3['geometry'].tolist()

# Criar os pesos da matriz de vizinhança queen
w_queen = Queen.from_iterable(geometrias)

# Adicionar uma coluna no DataFrame df3 com os valores dos pesos
df3['pesos_rainha'] = w_queen.weights

# Visualizar as primeiras linhas do DataFrame para verificar se a coluna 'pesos_rainha' foi adicionada corretamente
print(df3.head())

In [ ]:
# CRIANDO PESOS DA MATRIZ DE VIZINHANÇA QUEEN PARA OS ANOS DE 2010-2021
anos = ['2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021']

# Loop para calcular e adicionar os pesos para cada ano
for ano in anos:
    # Criar os pesos da matriz de vizinhança para o ano atual
    w_ano = Queen.from_dataframe(df3, geom_col='geometry', ids=df3.index.tolist())

    # Calcular os pesos baseados no ano atual e atribuir à lista de pesos da rainha
    w_ano.weights = [list(df3[ano][w_ano.neighbors[i]]) for i in range(len(w_ano.weights))]

    # Adicionar uma coluna no DataFrame df3 com os valores dos pesos do ano atual
    df3[f'w_{ano}'] = w_ano.weights

# Visualizar as primeiras linhas do DataFrame para verificar se as colunas foram adicionadas corretamente
print(df3.head())

In [ ]:
#MAPA MATRIZ DE VIZINHANÇA QUEEN
# Lista dos anos desejados
anos = ['2010', '2015', '2021']

# Definindo a opacidade mínima para o mapa de calor
min_opacity = 0.3

# Loop para criar mapas sequenciais para cada ano
for ano in anos:
    # Criar o título com o ano
    titulo = f'Análise Espacial por Matriz de Vizinhança Queen {ano}'

    # Criar o mapa centrado no Brasil
    m = folium.Map(location=[-14.235, -51.925], zoom_start=4)

    # Criar uma lista de tuplas contendo latitude, longitude e soma dos pesos do ano atual
    heat_data = [[row['geometry'].xy[1][0], row['geometry'].xy[0][0], sum(row[f'w_{ano}'])] for idx, row in df3.iterrows()]

    # Adicionar o mapa de calor com os dados do ano atual
    HeatMap(heat_data, min_opacity=min_opacity).add_to(m)

    # Extraindo os valores mínimos e máximos dos pesos para a escala de cores
    min_weight = min([row[2] for row in heat_data])
    max_weight = max([row[2] for row in heat_data])

    # Criar uma escala de cores de intensidade de verde a vermelho
    colormap = LinearColormap(colors=['green', 'yellow', 'red'], vmin=min_weight, vmax=max_weight)
    colormap.caption = f'Escala de Cores para w_{ano}: Verde (Menos Intenso) - Amarelo - Vermelho (Mais Intenso)'
    colormap.add_to(m)

    # Adicionar legenda ao mapa
    m.add_child(colormap)

    # Use o método display para exibir o título e o mapa no notebook
    display(HTML(f'<h3>{titulo}</h3>'))
    display(m)

A análise espacial de matriz de vizinhança queen demonstra que em 2010 a maior concentração de casos, estava no estado de Minas Gerais, sendo substituído por São Paulo em 2015 que passou a apresentar a maior quantidade de casos. Em 2021 podemos observar que MG novamente apresentou uma alta do numero de casos equiparando-se a São Paulo. Também é possivel ver áreas focais no localizadas nas regiões do nordeste e sul do país.

In [ ]:
#MODELO DE MAPA PREDITIVO PARA 2026 MATRIZ DE VIZINHANÇA QUEEN
# Definindo a opacidade mínima para o mapa de calor
min_opacity = 0.3

# Calcular os pesos para 2026
df3['w_2026'] = df3['w_2021']  # Aqui você pode inserir a lógica para calcular os pesos para 2026

# Criar o título para o mapa preditivo
titulo = 'Mapa Preditivo Matriz de Vizinhança queen para 2026'

# Criar o mapa centrado no Brasil
m = folium.Map(location=[-14.235, -51.925], zoom_start=4)

# Criar uma lista de tuplas contendo latitude, longitude e soma dos pesos para 2026
heat_data_2026 = [[row['geometry'].xy[1][0], row['geometry'].xy[0][0], sum(row['w_2026'])] for idx, row in df3.iterrows()]

# Adicionar o mapa de calor com os dados para 2026
HeatMap(heat_data_2026, min_opacity=min_opacity).add_to(m)

# Extraindo os valores mínimos e máximos dos pesos para a escala de cores
min_weight_2026 = min([row[2] for row in heat_data_2026])
max_weight_2026 = max([row[2] for row in heat_data_2026])

# Criar uma escala de cores de intensidade de verde a vermelho
colormap_2026 = LinearColormap(colors=['green', 'yellow', 'red'], vmin=min_weight_2026, vmax=max_weight_2026)
colormap_2026.caption = 'Escala de Cores para w_2026: Verde (Menos Intenso) - Amarelo - Vermelho (Mais Intenso)'
colormap_2026.add_to(m)

# Adicionar legenda ao mapa
m.add_child(colormap_2026)

# Use o método display para exibir o título e o mapa no notebook
display(HTML(f'<h3>{titulo}</h3>'))
display(m)

In [ ]:
#ANÁLISE DE MÉDIA MÓVEL PREDITIVO PARA 2026

# Calculando a média móvel com uma janela de tamanho 3
media_movel = df2[['ES', 'MG', 'SP', 'RJ']].rolling(window=3, axis=0).mean()

# Removendo os valores NaN resultantes da média móvel
media_movel.dropna(inplace=True)

# Adicionando uma coluna com os anos para usar como recurso para a regressão linear
media_movel['Ano_notificacao'] = df2.loc[media_movel.index, 'Ano_notificacao']

# Criando o modelo de regressão linear
model = LinearRegression()

# Plotando o gráfico de linhas para a média móvel
plt.figure(figsize=(10, 6))
for column in media_movel.columns[:-1]:  # Excluindo a coluna 'Ano_notificacao'
    # Separando os dados de entrada (X) e saída (y)
    X = media_movel[['Ano_notificacao']]
    y = media_movel[column]

    # Treinando o modelo
    model.fit(X, y)

    # Prevendo os valores para 2026
    predicao_2026 = model.predict([[2026]])

    print(f"Previsão de média móvel para 2026 ({column}): {predicao_2026[0]:.2f}")

    # Plotando a série temporal
    plt.plot(media_movel['Ano_notificacao'], media_movel[column], label=column)

    # Plotando linhas pontilhadas após 2021
    plt.plot([2021, 2026], [media_movel.iloc[-1][column], predicao_2026[0]], linestyle='--', color='gray')
    plt.scatter(2026, predicao_2026, color='red')  # Plotando o ponto da previsão para 2026

plt.title('Média Móvel por Ano e Estado')
plt.xlabel('Ano')
plt.ylabel('Média Móvel')
plt.xticks(rotation=45)
plt.grid(True)
plt.legend(title='Estado')
plt.tight_layout()
plt.show()

Os dois modelos de predição demonstram que em 2026 os casos de sífilis tenderão estar mais elevados que atualmente, a análise dos mapas, nos permite observar que São Paulo continuará apresentando mais casos de infecção para sífilis, enquanto as regiões de Minas Gerais e Rio de Janeiro apresentarão constância
 nos números de casos positivos, mas ainda assim, continuará em níveis elevados. Os estados da região sul do país, também deverão apresentar casos focais, especialmente na região de Porto Alegre e Curitiba. No nordeste, os principais pontos focais demonstram-se entre as regiões litorâneas dos estados do Ceará a Bahia.

**CONCLUSÃO**

Podemos concluir que indivíduos jovens (14-40 anos), que residem na região do sudeste, com ensino médio e 3º grau completos, possuem mais chances de contrair infecção por sífilis. Os dados também demonstram, tendência de aumento do número de casos, nos próximos anos. As análises geoespaciais, revelaram também, que existem regiões focais no nordeste e sul do país, enquanto o norte, é a região com menos casos notificados.